In [ ]:
###### lamda function

import json
import boto3
import urllib3
import datetime

# REPLACE WITH YOUR DATA FIREHOSE NAME
FIREHOSE_NAME = 'PUT-S3-y2B3Y'

def lambda_handler(event, context):

    http = urllib3.PoolManager()

    r = http.request("GET", "https://api.open-meteo.com/v1/forecast?latitude=42.0347&longitude=-93.6199&daily=temperature_2m_max,temperature_2m_min,daylight_duration,sunshine_duration,uv_index_max,precipitation_sum,rain_sum,wind_speed_10m_max&temperature_unit=fahrenheit&timezone=America%2FDenver&start_date=2024-03-01&end_date=2024-04-30")

    # turn it into a dictionary
    r_dict = json.loads(r.data.decode(encoding='utf-8', errors='strict'))

    time_list = []
    for val in r_dict['daily']['time']:
        time_list.append(val)

    temp_list1 = []
    for temp in r_dict['daily']['temperature_2m_max']:
        temp_list1.append(temp)

    temp_list2 = []
    for temp in r_dict['daily']['temperature_2m_min']:
        temp_list2.append(temp)

    duration_list1 = []
    for dur in r_dict['daily']['daylight_duration']:
        duration_list1.append(dur)

    duration_list2 = []
    for dur in r_dict['daily']['sunshine_duration']:
        duration_list2.append(dur)

    sum_list1 = []
    for total in r_dict['daily']['precipitation_sum']:
        sum_list1.append(total)

    sum_list2 = []
    for total in r_dict['daily']['rain_sum']:
        sum_list2.append(total)

    speed_list = []
    for speed in r_dict['daily']['wind_speed_10m_max']:
        speed_list.append(speed)

    # extract pieces of the dictionary
    processed_dict = {}

    # append to string running_msg
    running_msg = ''
    for i in range(len(time_list)):
        # construct each record
        processed_dict['latitude'] = r_dict['latitude']
        processed_dict['longitude'] = r_dict['longitude']
        processed_dict['time'] = time_list[i]
        processed_dict['temp_f'] = temp_list1[i]
        processed_dict['temp_f'] = temp_list2[i]
        processed_dict['row_ts'] = str(datetime.datetime.now())
        processed_dict['daylight_duration'] = duration_list1[i]
        processed_dict['sunshine_duration'] = duration_list2[i]
        processed_dict['precipitation_sum'] = sum_list1[i]
        processed_dict['rain_sum'] = sum_list2[i]
        processed_dict['wind_speed_10m_max'] = speed_list[i]

    # add a newline to denote the end of a record
    # add each record to the running_msg
        running_msg += str(processed_dict) + '\n'

    # cast to string
    running_msg = str(running_msg)
    fh = boto3.client('firehose')

    reply = fh.put_record_batch(
        DeliveryStreamName=FIREHOSE_NAME,
        Records = [
                 {'Data': running_msg}
    ]
    )

    return reply


###### Glue Jobs
###### delete parquet table job
import sys
import json
import boto3

# replace these with the names from your environment
BUCKET_TO_DEL = 'historical-weather-data-final-parquet-bucket'
DATABASE_TO_DEL = 'de_proj_database_hist_final'
TABLE_TO_DEL = 'historical_weather_data_final_parquet_tbl'
QUERY_OUTPUT_BUCKET = 's3://query-results-location-de-proj-3/'
# note: this script will only delete the parquet table, not bucket and database

# delete all objects in the bucket
s3_client = boto3.client('s3')

while True:
    objects = s3_client.list_objects(Bucket=BUCKET_TO_DEL)
    content = objects.get('Contents', [])
    if len(content) == 0:
        break
    for obj in content:
        s3_client.delete_object(Bucket=BUCKET_TO_DEL, Key=obj['Key'])


# drop the table too
client = boto3.client('athena')

queryStart = client.start_query_execution(
    QueryString = f"""
    DROP TABLE IF EXISTS {DATABASE_TO_DEL}.{TABLE_TO_DEL};
    """,
    QueryExecutionContext = {
        'Database': f'{DATABASE_TO_DEL}'
    },
    ResultConfiguration = { 'OutputLocation': f'{QUERY_OUTPUT_BUCKET}'}
)

# list of responses
resp = ["FAILED", "SUCCEEDED", "CANCELLED"]

# get the response
response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# wait until query finishes
while response["QueryExecution"]["Status"]["State"] not in resp:
    response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# if it fails, exit and give the Athena error message in the logs
if response["QueryExecution"]["Status"]["State"] == 'FAILED':
    sys.exit(response["QueryExecution"]["Status"]["StateChangeReason"])

###### create parquet table job

import sys
import boto3

client = boto3.client('athena')

SOURCE_TABLE_NAME = 'historical_weather_data_final' #the original table where crawler first stores data in athena
NEW_TABLE_NAME = 'historical_weather_data_final_parquet_tbl' # running this whole script will create this table in athena
NEW_TABLE_S3_BUCKET = 's3://historical-weather-data-final-parquet-bucket/' # you need to create this bucket manually in s3 beforehead
MY_DATABASE = 'de_proj_database_hist_final' # source table database
QUERY_RESULTS_S3_BUCKET = 's3://query-results-location-de-proj-3'  # no need to create fresh bucket for query results

# Refresh the table
queryStart = client.start_query_execution(
    QueryString = f"""
    CREATE TABLE {NEW_TABLE_NAME} WITH
    (external_location='{NEW_TABLE_S3_BUCKET}',
    format='PARQUET',
    write_compression='SNAPPY',
    partitioned_by = ARRAY['time'])
    AS

    SELECT
        latitude
        ,longitude
        ,temp_F
        ,(temp_F - 32) * (5.0/9.0) AS temp_C
        ,row_ts
        ,daylight_duration
        ,sunshine_duration
        ,precipitation_sum
        ,rain_sum
        ,wind_speed_10m_max
        ,time
    FROM "{MY_DATABASE}"."{SOURCE_TABLE_NAME}"

    ;
    """,
    QueryExecutionContext = {
        'Database': f'{MY_DATABASE}'
    },
    ResultConfiguration = { 'OutputLocation': f'{QUERY_RESULTS_S3_BUCKET}'}
)

# list of responses
resp = ["FAILED", "SUCCEEDED", "CANCELLED"]

# get the response
response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# wait until query finishes
while response["QueryExecution"]["Status"]["State"] not in resp:
    response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# if it fails, exit and give the Athena error message in the logs
if response["QueryExecution"]["Status"]["State"] == 'FAILED':
    sys.exit(response["QueryExecution"]["Status"]["StateChangeReason"])


###### data quality check job

import sys
import awswrangler as wr

# this check counts the number of NULL values in the response variable columns
# if any column contains NULL, the check returns a number > 0
NULL_DQ_CHECK = f"""
SELECT

    SUM(CASE WHEN temp_f IS NULL THEN 1 ELSE 0 END) AS temp_f_nulls,
    SUM(CASE WHEN temp_c IS NULL THEN 1 ELSE 0 END) AS temp_c_nulls,
    SUM(CASE WHEN daylight_duration IS NULL THEN 1 ELSE 0 END) AS daylight_duration_nulls,
    SUM(CASE WHEN sunshine_duration IS NULL THEN 1 ELSE 0 END) AS sunshine_duration_nulls,
    SUM(CASE WHEN precipitation_sum IS NULL THEN 1 ELSE 0 END) AS precipitation_sum_nulls,
    SUM(CASE WHEN rain_sum IS NULL THEN 1 ELSE 0 END) AS rain_sum_nulls,
    SUM(CASE WHEN wind_speed_10m_max IS NULL THEN 1 ELSE 0 END) AS wind_speed_10m_max_nulls

FROM "de_proj_database_hist_final"."historical_weather_data_final_parquet_tbl"
;
"""

# run the quality check
df = wr.athena.read_sql_query(sql=NULL_DQ_CHECK, database="de_proj_database_hist_final")

# exit if we get a result > 0
# else, the check was successful
if df['temp_f_nulls'][0] > 0 or df['temp_c_nulls'][0] > 0 or df['daylight_duration_nulls'][0] > 0 or df['sunshine_duration_nulls'][0] > 0 or df['precipitation_sum_nulls'][0] > 0 or df['rain_sum_nulls'][0] > 0 or df['wind_speed_10m_max_nulls'][0] > 0 :
    sys.exit('Results returned. Quality check failed.')
else:
    print('Quality check passed.')


###### publish production table job

import sys
import boto3
from datetime import datetime

QUERY_RESULTS_BUCKET = 's3://query-results-location-de-proj-3/' # same old bucket, not necessary to create new one
MY_DATABASE = 'de_proj_database_hist_final'
SOURCE_PARQUET_TABLE_NAME = 'historical_weather_data_final_parquet_tbl'
NEW_PROD_PARQUET_TABLE_NAME = 'historical_weather_data_final_prod_tbl' # this whole query will create this table with datetime in athena
NEW_PROD_PARQUET_TABLE_S3_BUCKET = 's3://historical-weather-data-final-prod-bucket' # need to create this bucket in s3 manually before run

# create a string with the current UTC datetime
# convert all special characters to underscores
# this will be used in the table name and in the bucket path in S3 where the table is stored
DATETIME_NOW_INT_STR = str(datetime.now()).replace('-', '_').replace(' ', '_').replace(':', '_').replace('.', '_')

client = boto3.client('athena')

# Refresh the table
queryStart = client.start_query_execution(
    QueryString = f"""
    CREATE TABLE {NEW_PROD_PARQUET_TABLE_NAME}_{DATETIME_NOW_INT_STR} WITH
    (external_location='{NEW_PROD_PARQUET_TABLE_S3_BUCKET}/{DATETIME_NOW_INT_STR}/',
    format='PARQUET',
    write_compression='SNAPPY',
    partitioned_by = ARRAY['time'])
    AS

    SELECT
        *
    FROM "{MY_DATABASE}"."{SOURCE_PARQUET_TABLE_NAME}"

    ;
    """,
    QueryExecutionContext = {
        'Database': f'{MY_DATABASE}'
    },
    ResultConfiguration = { 'OutputLocation': f'{QUERY_RESULTS_BUCKET}'}
)

# list of responses
resp = ["FAILED", "SUCCEEDED", "CANCELLED"]

# get the response
response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# wait until query finishes
while response["QueryExecution"]["Status"]["State"] not in resp:
    response = client.get_query_execution(QueryExecutionId=queryStart["QueryExecutionId"])

# if it fails, exit and give the Athena error message in the logs
if response["QueryExecution"]["Status"]["State"] == 'FAILED':
    sys.exit(response["QueryExecution"]["Status"]["StateChangeReason"])



############################################################# End ######################################

